# Описание проекта и целей


Необходимо было провести анализ рынка труда в России по вакансиям на hh.ru по запросам:
- Аналитик данных
- Бизнес Аналитик

Целью работы является выявление различий в предлагаемых вакансиях.

Необходимо проверить следующие гипотезы:

- Большую долю вакансий занимают вакансии на позицию Junior с опытом от года.
- Москва лидирует по количеству вакансий.
- Предложений на удаленную работу больше чем на работу в офисе.

Ход исследования

Данные для анализа мы получим по следующей ссылке https://disk.yandex.ru/d/qVTeq4KBlHdA7g. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных и скорей всего предобработка.

Таким образом, исследование пройдёт в четыре этапа:

1. Обзор данных.
2. Предобработка данных.
3. Исследовательский анализ.
4. Выводы по проекту и проверка гипотез.



# Обзор данных

In [1]:
# импортируем необходимые библиотеки
!pip install plotly
!pip install openpyxl
import time
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from plotly import graph_objects as go
%matplotlib inline

from datetime import datetime
from datetime import date, timedelta as td
from datetime import datetime, timedelta
# для визуализации
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)
warnings.filterwarnings(action='ignore')

In [2]:
# загружаем данные 
data_da = pd.read_excel('/Users/kseniaadzic/Downloads/vacancies_da.xlsx')
display(data_da.info())
data_ba = pd.read_excel('/Users/kseniaadzic/Downloads/vacancies_ba.xlsx')
display(data_ba.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   id                                417 non-null    int64         
 1   name                              417 non-null    object        
 2   published_at                      417 non-null    datetime64[ns]
 3   alternate_url                     417 non-null    object        
 4   type                              417 non-null    object        
 5   employer                          417 non-null    object        
 6   department                        137 non-null    object        
 7   area                              417 non-null    object        
 8   experience                        417 non-null    object        
 9   key_skills                        417 non-null    object        
 10  schedule                          417 non-null    

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   id                                1011 non-null   int64         
 1   name                              1011 non-null   object        
 2   published_at                      1011 non-null   datetime64[ns]
 3   alternate_url                     1011 non-null   object        
 4   type                              1011 non-null   object        
 5   employer                          1011 non-null   object        
 6   department                        210 non-null    object        
 7   area                              1011 non-null   object        
 8   experience                        1011 non-null   object        
 9   key_skills                        1011 non-null   object        
 10  schedule                          1011 non-null 

None

In [3]:
# обработаем 1-ый датафрейм 
data_da.shape


(417, 20)

In [4]:
# преобразуем столбец name и удалим строки со словами системный и bi
data_da.name = data_da.name.str.lower()
data_da.name = data_da.name.str.strip()
data_da.name = data_da.name.str.replace('-',' ')
data_da.name = data_da.name.str.replace('/', ' ')
data_da = data_da[~data_da.name.str.contains('системный') & ~data_da.name.str.contains('bi')]
data_da.shape


(375, 20)

In [5]:
# преобразуем столбец name и удалим строки со словами системный и bi во втором датафрейме
data_ba.name = data_ba.name.str.lower()
data_ba.name = data_ba.name.str.strip()
data_ba.name = data_ba.name.str.replace('-',' ')
data_ba.name = data_ba.name.str.replace('/', ' ')
data_ba = data_ba[~data_ba.name.str.contains('системный') & ~data_ba.name.str.contains('bi')]
#data_ba.name.value_counts() 
data_ba.shape
#data_ba.isna().sum()

(925, 20)

In [7]:
# соединим два датафрейма
df = pd.concat([data_da, data_ba], axis=0)
df.shape


(1300, 20)

объединение прошло успешно 

In [8]:
# создадим функцию для получения общей информации по датафрейму
def df_review(df):
    start = "\033[1m"
    end2 = "\033[0;0m"
    print(start + f'Вывод инфо' + end2, end='\n\n')
    print(df.info())
    print('---------------------------------------------------------------------------------------------------------')
    print(start + f'Вывод описания' + end2, end='\n\n')
    print(df.describe())
    print('---------------------------------------------------------------------------------------------------------')
    print(start + f'Количество пропусков' + end2, end='\n\n')
    print(df.isna().sum())
    print('---------------------------------------------------------------------------------------------------------')
    print(start + f'Количество пропусков в процентах' + end2, end='\n\n')
    display(pd.DataFrame(round(df.isna().mean()*100,)).style.background_gradient('coolwarm'))
    print('---------------------------------------------------------------------------------------------------------')
    print(start + f'Количество дублей' + end2, end='\n\n')
    print(df.duplicated().sum())

In [9]:
df_review(df)

Вывод инфо

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1300 entries, 0 to 1010
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   id                                1300 non-null   int64         
 1   name                              1300 non-null   object        
 2   published_at                      1300 non-null   datetime64[ns]
 3   alternate_url                     1300 non-null   object        
 4   type                              1300 non-null   object        
 5   employer                          1300 non-null   object        
 6   department                        313 non-null    object        
 7   area                              1300 non-null   object        
 8   experience                        1300 non-null   object        
 9   key_skills                        1300 non-null   object        
 10  schedule                          13

,0
id,0.000000
name,0.000000
published_at,0.000000
alternate_url,0.000000
type,0.000000
employer,0.000000
department,76.000000
area,0.000000
experience,0.000000
key_skills,0.000000


---------------------------------------------------------------------------------------------------------
Количество дублей

16


**Выводы по обзору данных:**
1. Вначале мы загрузили два датасета с вакансиями аналитик данных и бизнес аналитик;
2. Сразу удалили строки со словами 'системный' и 'bi', так как эти направления нас не интересуют;
3. Далее объединили два датафрейма в один.
4. В новом датафрейме 1300 вакансий с 20 характеристиками.
5. Имеются пропуски в категориальных данных, а также в зарплате.
6. Выявлено 16 явных дубликатов.

Приступим к предобработке данных.

# Предобработка данных 

In [10]:
# изменим типы данных где это необходимо
df.id = df.id.astype(object)
df.published_at = pd.to_datetime(df.published_at).dt.date

df.head()


,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,description_lemmatized,salary_from,salary_to,salary_bin,key_skills_from_key_skills_field,hard_skills_from_description,soft_skills_from_description
0,77320677,управляющий эксперт по аналитике данных,2024-03-18,https://hh.ru/vacancy/77320677,Открытая,ПСБ (ПАО «Промсвязьбанк»),"ПАО «Промсвязьбанк», ИТ",Москва,Middle (3-6 years),"['Бизнес-анализ', 'Анализ бизнес показателей',...",Полный день,Полная занятость,на масштабный проект по созданию единого корпо...,масштабный проект создание единый корпоративны...,NaN,NaN,ЗП не указана,"Финансовый анализ, Бизнес-анализ, Финансовая о...",NaN,Документация
2,79889558,аналитик данных в группу управления метаданным...,2024-03-16,https://hh.ru/vacancy/79889558,Открытая,МТС,"«МТС», Digital",Москва,Junior+ (1-3 years),"['Data Catalog', 'Data Governance', 'DMBOK DAM...",Полный день,Полная занятость,мтс – это мультисервисная цифровая экосистема....,мтс мультисервисный цифровой экосистема создав...,NaN,NaN,ЗП не указана,"Data Catalog, DMBOK DAMA, Business Glossary, D...",NaN,NaN
3,82901165,data аналитик,2024-03-25,https://hh.ru/vacancy/82901165,Открытая,ЭлРос,NaN,Владимир,Junior+ (1-3 years),"['Spark', 'Data Analysis', 'Теория вероятносте...",Полный день,Полная занятость,наши преимущества: аккредитованная ит-компан...,наш преимущество аккредитовать ита компания си...,NaN,NaN,ЗП не указана,"Python, Математическая статистика, SQL, Spark,...",NaN,NaN
7,87684971,аналитик данных,2024-03-18,https://hh.ru/vacancy/87684971,Открытая,Кадровый Центр Департамента здравоохранения го...,NaN,Москва,Junior+ (1-3 years),"['Аналитическое мышление', 'MS Excel', 'MS Pow...",Полный день,Полная занятость,обязанности: анализ данных по заданным критер...,обязанность анализ данные заданный критерий по...,100000.00,NaN,ЗП не указана,"MS Excel, Аналитика, MS PowerPoint, Анализ дан...",NaN,NaN
8,88605313,data analyst,2024-03-12,https://hh.ru/vacancy/88605313,Открытая,Бланк – банк для бизнеса,NaN,Москва,Junior+ (1-3 years),"['Power BI', 'PowerBI', 'Tableau', 'PostgreSQL...",Удаленная работа,Полная занятость,"привет!мы – бланк, современный life-first моби...",привет бланк современный life first мобильный ...,NaN,NaN,ЗП не указана,"SQL, Python, Power BI, PowerBI, DAX, MS Power ...",NaN,NaN


In [11]:
# удалим дубликаты
df = df.drop_duplicates()
df.duplicated().sum()

0

In [12]:
df.columns


Index(['id', 'name', 'published_at', 'alternate_url', 'type', 'employer',
       'department', 'area', 'experience', 'key_skills', 'schedule',
       'employment', 'description', 'description_lemmatized', 'salary_from',
       'salary_to', 'salary_bin', 'key_skills_from_key_skills_field',
       'hard_skills_from_description', 'soft_skills_from_description'],
      dtype='object')

Для удобства создадим столбец, который будет содержать наименование вакансии(либо аналитик данных либо бизнес аналитик)

In [13]:
# создадим новый столбец, содержащий наименование вакансии 
df['name'] = df['name'].str.replace('business', 'бизнес')
df['category'] = np.where(df['name'].str.contains('бизнес'), 'бизнес аналитик', 'аналитик данных')
df['category'].value_counts()


бизнес аналитик    926
аналитик данных    358
Name: category, dtype: int64

In [14]:
print('Из {} вакансий величина ЗП указана в {} случаях ({:.2%})'.format(len(df), df['salary_to'].count(), df['salary_to'].count()/len(df)))
print('Из {} вакансий ключевые навыки указаны в {} случаях ({:.2%})'.format(len(df), df['key_skills_from_key_skills_field'].count(), df['key_skills_from_key_skills_field'].count()/len(df)))
print('Всего компаний в выгрузке: {}'.format(len(df['employer'].unique())))

Из 1284 вакансий величина ЗП указана в 142 случаях (11.06%)
Из 1284 вакансий ключевые навыки указаны в 701 случаях (54.60%)
Всего компаний в выгрузке: 640


Заполним пропуски в категориальных столбцах словом 'неизвестно', в столбцах о зарплате пропуски составляют 89%. Принято решение заполнить 0, так как медианными значениями заполнять не имеет смысла, будет сильное искажение данных. 

In [15]:
# заполним пропуски в датафрайме
df.key_skills_from_key_skills_field = df.key_skills_from_key_skills_field.fillna('неизвестно')
df.department = df.department.fillna('неизвестно')
df.salary_from = df.salary_from.fillna(0)
df.salary_to = df.salary_to.fillna(0)
df.hard_skills_from_description = df.hard_skills_from_description.fillna('неизвестно')
df.soft_skills_from_description = df.soft_skills_from_description.fillna('неизвестно')
#проверим 
df.isna().sum()

id                                  0
name                                0
published_at                        0
alternate_url                       0
type                                0
employer                            0
department                          0
area                                0
experience                          0
key_skills                          0
schedule                            0
employment                          0
description                         0
description_lemmatized              0
salary_from                         0
salary_to                           0
salary_bin                          0
key_skills_from_key_skills_field    0
hard_skills_from_description        0
soft_skills_from_description        0
category                            0
dtype: int64

Далее приступим к дальнейшей предообработке столбцов датафрейма

In [16]:
#проверим стобец type и приведем к нижнему регистру и удалим лишние пробелы
df.type = df.type.str.lower()
df.type = df.type.str.strip()
df.type.value_counts()


открытая    1284
Name: type, dtype: int64

In [17]:
# столбец employer приведем к нижнему регистру и удалим лишние пробелы
df.employer = df.employer.str.lower()
df.employer = df.employer.str.strip()
df.employer.value_counts().head(15)


сбер                         95
магнит, розничная сеть       30
банк втб (пао)               24
т1                           23
ozon                         15
мтс                          14
комус                        14
aston                        11
axenix (ранее accenture)     11
правительство москвы         11
wildberries                  11
совкомбанк                   10
технологии доверия           10
газпром нефть                 9
псб (пао «промсвязьбанк»)     9
Name: employer, dtype: int64

In [18]:
df.department = df.department.str.lower()
df.department = df.department.str.strip()
df.department.value_counts().head(15)

неизвестно                                      974
сбер для экспертов                               83
гк иннотех | финтех (иннотех)                    13
пао втб, средний и малый бизнес                  12
ozon офис и коммерция                             9
«мтс»                                             9
технологии доверия, ит и цифровые технологии      8
сбер. it                                          8
газпромбанк                                       8
пао «промсвязьбанк», бизнес и инфраструктура      7
пао «газпром нефть» ит                            6
яндекс                                            6
северсталь. it & digital                          6
гк иннотех | большие данные                       6
мегафон, офис                                     6
Name: department, dtype: int64

In [19]:
df.area = df.area.str.lower()
df.area = df.area.str.strip()
df.area.value_counts().head(15)

москва             801
санкт-петербург    123
екатеринбург        33
нижний новгород     29
казань              28
краснодар           27
самара              17
новосибирск         17
ростов-на-дону      13
владивосток         10
пермь                9
воронеж              9
тюмень               7
саратов              7
владимир             6
Name: area, dtype: int64

In [20]:
df.experience = df.experience.str.lower()
df.experience = df.experience.str.strip()

df.experience.value_counts()

junior+ (1-3 years)      675
middle (3-6 years)       505
junior (no experince)     92
senior (6+ years)         12
Name: experience, dtype: int64

In [21]:
df.key_skills = df.key_skills.str.strip()
df.key_skills = df.key_skills.str.lower()

df.key_skills.value_counts().head(15)

[]                                                                                                                                                                                                                                                                                                                                                259
['документация']                                                                                                                                                                                                                                                                                                                                  112
['документация', 'confluence']                                                                                                                                                                                                                                                                                              

In [22]:
df.schedule = df.schedule.str.lower()
df.schedule = df.schedule.str.strip()
df.schedule.value_counts()

полный день         1045
удаленная работа     204
гибкий график         32
сменный график         3
Name: schedule, dtype: int64

In [23]:
df.employment = df.employment.str.lower()
df.employment = df.employment.str.strip()
df.employment.value_counts()


полная занятость       1255
стажировка               19
частичная занятость       6
проектная работа          4
Name: employment, dtype: int64

In [24]:
df.description = df.description.str.lower()
df.description = df.description.str.strip()
df.description.value_counts().head(2)

компания axenix (ранее accenture) продолжает работу на российском рынке и аккумулирует 30-ти летний консалтинговый опыт внедрения инновационных решений.наша экспертиза - стратегия и консалтинг, технологии и операции, направленные на цифровизацию бизнеса. в своей деятельности мы сочетаем обширные знания, опыт в различных отраслях и глубокое понимание специфики и возможностей российского бизнеса. офисы компании находятся в москве, твери и ростове-на-дону, алматы и ереване.помимо этого, у нас есть команды в санкт-петербурге, краснодаре и воронеже, а также сотрудники, работающие удаленно из разных регионов.обязанности:  помощь в предоставлении стратегических консультаций и реализации проектов по трансформации бизнеса с помощью инновационных процессов и решений в области данных и искусственного интеллекта (ключевые индустрии – телеком, it, гос сектор) проведение исследований рынка больших данных и искусственного интеллекта, стратегических подходов к регулированию сферы и подходов к управлен

In [25]:
df.description_lemmatized = df.description_lemmatized.str.lower()  
df.description_lemmatized = df.description_lemmatized.str.strip()  

df.description_lemmatized.value_counts().head(2)   


компания axenix ранее accenture продолжать работа российский рынок аккумулировать 3 ти летний консалтинговый опыт внедрение инновационный решение наш экспертиза стратегия консалтинг технология операция направлять цифровизация бизнес свой деятельность сочетать обширный знание опыт различный отрасль глубокий понимание специфика возможность российский бизнес офис компания находиться москва тверь ростов дон алматы ереван помимо команда санкт петербург краснодар воронеж также сотрудник работать удаленный разный регион обязанность помощь предоставление стратегический консультация реализация проект трансформация бизнес помощь инновационный процесс решение область данные искусственный интеллект ключевой индустрия телеком it гос сектор проведение исследование рынок большой данные искусственный интеллект стратегический подход регулирование сфера подход управление данные монетизация данные различный страна индустрия участие разработка расчет защита коммерческий предложение клиент сбор анализ форм

In [26]:

df.salary_from.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


0.00         83.6%
100000.00     2.6%
70000.00      1.7%
150000.00     1.2%
50000.00      1.2%
120000.00     1.2%
200000.00     0.8%
80000.00      0.7%
130000.00     0.5%
60000.00      0.5%
180000.00     0.5%
140000.00     0.4%
90000.00      0.4%
250000.00     0.4%
300000.00     0.3%
75000.00      0.3%
110000.00     0.3%
170000.00     0.2%
85000.00      0.2%
52000.00      0.2%
160000.00     0.2%
137000.00     0.2%
30000.00      0.2%
35000.00      0.2%
49000.00      0.1%
39000.00      0.1%
138000.00     0.1%
230000.00     0.1%
74600.00      0.1%
89000.00      0.1%
100328.00     0.1%
168000.00     0.1%
187000.00     0.1%
55215.00      0.1%
65000.00      0.1%
45000.00      0.1%
57500.00      0.1%
84000.00      0.1%
280000.00     0.1%
78439.00      0.1%
57000.00      0.1%
105000.00     0.1%
40050.00      0.1%
78200.00      0.1%
55000.00      0.1%
40000.00      0.1%
73000.00      0.1%
42500.00      0.1%
28940.00      0.1%
76000.00      0.1%
78045.00      0.1%
280500.00     0.1%
350000.00   

In [27]:
df.salary_to.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'


0.00         88.9%
150000.00     1.3%
200000.00     1.0%
180000.00     0.9%
80000.00      0.8%
100000.00     0.6%
110000.00     0.6%
300000.00     0.5%
250000.00     0.5%
220000.00     0.3%
400000.00     0.2%
350000.00     0.2%
230000.00     0.2%
280000.00     0.2%
73000.00      0.2%
160000.00     0.2%
170000.00     0.2%
70000.00      0.2%
50000.00      0.2%
120000.00     0.2%
85000.00      0.2%
40000.00      0.2%
55000.00      0.2%
130000.00     0.2%
78200.00      0.1%
63000.00      0.1%
60000.00      0.1%
207000.00     0.1%
140000.00     0.1%
137000.00     0.1%
374000.00     0.1%
81379.00      0.1%
233750.00     0.1%
135000.00     0.1%
98000.00      0.1%
172000.00     0.1%
78045.00      0.1%
270000.00     0.1%
58500.00      0.1%
195000.00     0.1%
218400.00     0.1%
115000.00     0.1%
145931.00     0.1%
81000.00      0.1%
93750.00      0.1%
36175.00      0.1%
125000.00     0.1%
67629.00      0.1%
217000.00     0.1%
Name: salary_to, dtype: object

In [28]:
df.salary_bin = df.salary_bin.str.lower()
df.salary_bin = df.salary_bin.str.strip()
df.salary_bin.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

зп не указана                88.9%
от 100 тысяч до 200 тысяч     4.8%
от 200 тысяч до 300 тысяч     2.6%
меньше 100 тысяч              2.6%
больше 300 тысяч              1.1%
Name: salary_bin, dtype: object

In [29]:
df.key_skills_from_key_skills_field = df.key_skills_from_key_skills_field.str.lower()
df.key_skills_from_key_skills_field = df.key_skills_from_key_skills_field.str.strip()
df.key_skills_from_key_skills_field = df.key_skills_from_key_skills_field.str.replace('-',' ')
df.key_skills_from_key_skills_field = df.key_skills_from_key_skills_field.str.replace('/', ' ')


 
           

In [30]:
df.hard_skills_from_description = df.hard_skills_from_description.str.lower()
df.hard_skills_from_description = df.hard_skills_from_description.str.strip()
df.hard_skills_from_description = df.hard_skills_from_description.str.replace('-',' ')
df.hard_skills_from_description = df.hard_skills_from_description.str.replace('/', ' ')
df.hard_skills_from_description.value_counts()


неизвестно               1052
confluence                159
pandas                     59
юнит экономика              9
pandas, confluence          3
pandas, google sheets       1
google sheets               1
Name: hard_skills_from_description, dtype: int64

In [31]:
# создадим функцию для преобразования списка списков в плоский список
def rad(col):
    result = []
    for element in col:
        if type(element) is list:
            for item in element:
                result.append(item)
        else:
            result.append(element)
    return result

 
df.key_skills_from_key_skills_field = rad(df.key_skills_from_key_skills_field)

df.head()


,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,description_lemmatized,salary_from,salary_to,salary_bin,key_skills_from_key_skills_field,hard_skills_from_description,soft_skills_from_description,category
0,77320677,управляющий эксперт по аналитике данных,2024-03-18,https://hh.ru/vacancy/77320677,открытая,псб (пао «промсвязьбанк»),"пао «промсвязьбанк», ит",москва,middle (3-6 years),"['бизнес-анализ', 'анализ бизнес показателей',...",полный день,полная занятость,на масштабный проект по созданию единого корпо...,масштабный проект создание единый корпоративны...,0.00,0.00,зп не указана,"финансовый анализ, бизнес анализ, финансовая о...",неизвестно,Документация,аналитик данных
2,79889558,аналитик данных в группу управления метаданным...,2024-03-16,https://hh.ru/vacancy/79889558,открытая,мтс,"«мтс», digital",москва,junior+ (1-3 years),"['data catalog', 'data governance', 'dmbok dam...",полный день,полная занятость,мтс – это мультисервисная цифровая экосистема....,мтс мультисервисный цифровой экосистема создав...,0.00,0.00,зп не указана,"data catalog, dmbok dama, business glossary, d...",неизвестно,неизвестно,аналитик данных
3,82901165,data аналитик,2024-03-25,https://hh.ru/vacancy/82901165,открытая,элрос,неизвестно,владимир,junior+ (1-3 years),"['spark', 'data analysis', 'теория вероятносте...",полный день,полная занятость,наши преимущества: аккредитованная ит-компан...,наш преимущество аккредитовать ита компания си...,0.00,0.00,зп не указана,"python, математическая статистика, sql, spark,...",неизвестно,неизвестно,аналитик данных
7,87684971,аналитик данных,2024-03-18,https://hh.ru/vacancy/87684971,открытая,кадровый центр департамента здравоохранения го...,неизвестно,москва,junior+ (1-3 years),"['аналитическое мышление', 'ms excel', 'ms pow...",полный день,полная занятость,обязанности: анализ данных по заданным критер...,обязанность анализ данные заданный критерий по...,100000.00,0.00,зп не указана,"ms excel, аналитика, ms powerpoint, анализ дан...",неизвестно,неизвестно,аналитик данных
8,88605313,data analyst,2024-03-12,https://hh.ru/vacancy/88605313,открытая,бланк – банк для бизнеса,неизвестно,москва,junior+ (1-3 years),"['power bi', 'powerbi', 'tableau', 'postgresql...",удаленная работа,полная занятость,"привет!мы – бланк, современный life-first моби...",привет бланк современный life first мобильный ...,0.00,0.00,зп не указана,"sql, python, power bi, powerbi, dax, ms power ...",неизвестно,неизвестно,аналитик данных


In [32]:
df.soft_skills_from_description = df.soft_skills_from_description.str.lower()
df.soft_skills_from_description = df.soft_skills_from_description.str.strip()

df.soft_skills_from_description.value_counts().head(15)

неизвестно                                            620
документация                                          340
коммуникация                                           76
аналитическое мышление                                 54
коммуникация, документация                             54
документация, аналитическое мышление                   36
документация, переговоры                               20
переговоры                                             16
коммуникация, документация, аналитическое мышление     10
проактивность                                           9
креативность                                            5
коммуникация, аналитическое мышление                    4
документация, проактивность                             4
документация, креативность                              3
коммуникация, проактивность                             3
Name: soft_skills_from_description, dtype: int64

In [33]:
df.columns

Index(['id', 'name', 'published_at', 'alternate_url', 'type', 'employer',
       'department', 'area', 'experience', 'key_skills', 'schedule',
       'employment', 'description', 'description_lemmatized', 'salary_from',
       'salary_to', 'salary_bin', 'key_skills_from_key_skills_field',
       'hard_skills_from_description', 'soft_skills_from_description',
       'category'],
      dtype='object')

**Выводы по предобработке:**

1. Были удалены явные дубликаты 
2. Все данные приведены к нижнему регистру и удалены лишние пробелы.
3. Пропуски в категориальных данных заполнены словом 'неизвестно', в зарплате пропуски было решено заполнить значением 0, так как только 11% данных имеют данные по зарплате.
4. Был создан столбец 'category', который мы заполнили общими названиями вакансий(аналитик данных и бизнес аналитик) из столбца 'name'.
5. Столбец 'key_skills_from_key_skills_field' имел тип список списков, поэтому для облегчения работы с ним, мы создали функцию для изменения списка списков в плоский список.
6. Столбец 'published_at' был приведен к формату date.
7. Данные все готовы для дальнейшего анализа.

# Исследовательский анализ 

Вначале предполагается провести исследовательский анализ данных, и потом в выводах осуществить проверку выдвинутых гипотез. 

## ТОП ключевых навыков требуемых в вакансиях Аналитик данных и Бизнес аналитик

In [35]:
df['skills_list'] = df['key_skills_from_key_skills_field'].apply(lambda x: x.split(', '))
df

skills_list = []
skills_data_analyst = []
skills_business_analyst = []

lists = {
    'аналитик данных':skills_data_analyst,
    'бизнес аналитик':skills_business_analyst
}


for keyword in lists:
    for skills in df.query('category==@keyword')['skills_list']:
        for skill in skills:
            if skill!='неизвестно':
                lists[keyword].append(skill)

skills_list = skills_data_analyst + skills_business_analyst
  

skills_data_analyst_df = pd.DataFrame({'skill':skills_data_analyst})
skills_business_analyst_df = pd.DataFrame({'skill':skills_business_analyst})
skills = pd.DataFrame({'skill':skills_list})

skills_data_analyst_df = pd.DataFrame(skills_data_analyst_df['skill'].value_counts()).reset_index()
skills_business_analyst_df = pd.DataFrame(skills_business_analyst_df['skill'].value_counts()).reset_index()
skills = pd.DataFrame(skills['skill'].value_counts()).reset_index()

skills_data_analyst_df.columns = ['skill', 'vacancies']
skills_business_analyst_df.columns = ['skill', 'vacancies']
skills.columns = ['skill', 'vacancies']

skills_data_analyst_df['percent_of_vacancies'] = skills_data_analyst_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and category=="аналитик данных"'))
skills_business_analyst_df['percent_of_vacancies'] = skills_business_analyst_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and category=="бизнес аналитик"'))
skills['percent_of_vacancies'] = skills['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно"'))

lists_for_plots = {
    'аналитик данных':skills_data_analyst_df,
    'бизнес аналитик':skills_business_analyst_df, 
    'аналитик данных / бизнес аналитик':skills
}

for keyword in lists_for_plots:
    fig = px.bar(lists_for_plots[keyword].head(15), x='skill', y='vacancies', width = 950, height = 450, color='skill', text='vacancies')
    fig.update_xaxes(tickangle=30)
    fig.update_layout(
        title='ТОП-15 ключевых навыков в профессии {} '.format(keyword),
        xaxis_title="Ключевой навык",
        yaxis_title="Вакансий с навыком")
    fig.update_traces(textposition='outside')
    for trace, percent in zip(fig.data, lists_for_plots[keyword].head(15)['percent_of_vacancies']):
        trace.name = trace.name.split('=')[0] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'


    fig.show()

## ТОП 15 ключевых навыков у Аналитика данных и Бизнес аналитика в разрезе требуемого опыта  

In [36]:
df['skills_list'] = df['key_skills_from_key_skills_field'].apply(lambda x: x.split(', '))
df

skills_list = []
skills_junior_1_3_y = []
skills_middle_3_6_y = []
skills_junior_no_exp = []
skills_senior = []
lists = {
    'junior+ (1-3 years)':skills_junior_1_3_y,
    'middle (3-6 years)':skills_middle_3_6_y,
    'junior (no experince)':skills_junior_no_exp,
    'senior (6+ years)':skills_senior

}


for keyword in lists:
    for skills in df.query('experience==@keyword and category=="аналитик данных"')['skills_list']:
        for skill in skills:
            if skill!='неизвестно':
                lists[keyword].append(skill)

skills_list = skills_junior_1_3_y + skills_middle_3_6_y + skills_junior_no_exp + skills_senior
  

skills_junior_1_3_y_df = pd.DataFrame({'skill':skills_junior_1_3_y})
skills_middle_3_6_y_df = pd.DataFrame({'skill':skills_middle_3_6_y})
skills_junior_no_exp_df = pd.DataFrame({'skill':skills_junior_no_exp})
skills_senior_df = pd.DataFrame({'skill':skills_senior})
skills = pd.DataFrame({'skill':skills_list})

skills_junior_1_3_y_df = pd.DataFrame(skills_junior_1_3_y_df['skill'].value_counts()).reset_index()
skills_middle_3_6_y_df = pd.DataFrame(skills_middle_3_6_y_df['skill'].value_counts()).reset_index()
skills_junior_no_exp_df = pd.DataFrame(skills_junior_no_exp_df['skill'].value_counts()).reset_index()
skills_senior_df = pd.DataFrame(skills_senior_df['skill'].value_counts()).reset_index()
skills = pd.DataFrame(skills['skill'].value_counts()).reset_index()

skills_junior_1_3_y_df.columns = ['skill', 'vacancies']
skills_middle_3_6_y_df.columns = ['skill', 'vacancies']
skills_junior_no_exp_df.columns = ['skill', 'vacancies']
skills_senior_df.columns = ['skill', 'vacancies']
skills.columns = ['skill', 'vacancies']

skills_junior_1_3_y_df['percent_of_vacancies'] = skills_junior_1_3_y_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and experience == "junior+ (1-3 years)"'))
skills_middle_3_6_y_df['percent_of_vacancies'] = skills_middle_3_6_y_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and experience=="middle (3-6 years)"'))       
skills_junior_no_exp_df['percent_of_vacancies'] = skills_junior_no_exp_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and experience=="junior (no experince)"'))
skills_senior_df['percent_of_vacancies'] = skills_senior_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and experience=="senior (6+ years)"'))
skills['percent_of_vacancies'] = skills['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно"'))


lists_for_plots = {
    'junior+ (1-3 years)':skills_junior_1_3_y_df,
    'middle (3-6 years)':skills_middle_3_6_y_df, 
    'junior (no experince)':skills_junior_no_exp_df,
    'senior (6+ years)':skills_senior_df
}
for keyword in lists_for_plots:
    fig = px.bar(lists_for_plots[keyword].head(15), x='skill', y='vacancies', width = 950, height = 450, color='skill', text='vacancies')
    fig.update_xaxes(tickangle=30)
    fig.update_layout(
        title='ТОП-15 ключевых навыков в профессии аналитик данных у {} '.format(keyword),
        xaxis_title="Ключевые навыки",
        yaxis_title="Количество вакансий с данным навыком")
    fig.update_traces(textposition='outside')
    for trace, percent in zip(fig.data, lists_for_plots[keyword].head(15)['percent_of_vacancies']):
        trace.name = trace.name.split('=')[0] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'


    fig.show()

In [37]:

df['skills_list'] = df['key_skills_from_key_skills_field'].apply(lambda x: x.split(', '))
df

skills_list = []
skills_junior_1_3_y = []
skills_middle_3_6_y = []
skills_junior_no_exp = []
skills_senior = []
lists = {
    'junior+ (1-3 years)':skills_junior_1_3_y,
    'middle (3-6 years)':skills_middle_3_6_y,
    'junior (no experince)':skills_junior_no_exp,
    'senior (6+ years)':skills_senior

}


for keyword in lists:
    for skills in df.query('experience==@keyword and category=="бизнес аналитик"')['skills_list']:
        for skill in skills:
            if skill!='неизвестно':
                lists[keyword].append(skill)

skills_list = skills_junior_1_3_y + skills_middle_3_6_y + skills_junior_no_exp + skills_senior
  

skills_junior_1_3_y_df = pd.DataFrame({'skill':skills_junior_1_3_y})
skills_middle_3_6_y_df = pd.DataFrame({'skill':skills_middle_3_6_y})
skills_junior_no_exp_df = pd.DataFrame({'skill':skills_junior_no_exp})
skills_senior_df = pd.DataFrame({'skill':skills_senior})
skills = pd.DataFrame({'skill':skills_list})

skills_junior_1_3_y_df = pd.DataFrame(skills_junior_1_3_y_df['skill'].value_counts()).reset_index()
skills_middle_3_6_y_df = pd.DataFrame(skills_middle_3_6_y_df['skill'].value_counts()).reset_index()
skills_junior_no_exp_df = pd.DataFrame(skills_junior_no_exp_df['skill'].value_counts()).reset_index()
skills_senior_df = pd.DataFrame(skills_senior_df['skill'].value_counts()).reset_index()
skills = pd.DataFrame(skills['skill'].value_counts()).reset_index()

skills_junior_1_3_y_df.columns = ['skill', 'vacancies']
skills_middle_3_6_y_df.columns = ['skill', 'vacancies']
skills_junior_no_exp_df.columns = ['skill', 'vacancies']
skills_senior_df.columns = ['skill', 'vacancies']
skills.columns = ['skill', 'vacancies']

skills_junior_1_3_y_df['percent_of_vacancies'] = skills_junior_1_3_y_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and experience == "junior+ (1-3 years)"'))
skills_middle_3_6_y_df['percent_of_vacancies'] = skills_middle_3_6_y_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and experience=="middle (3-6 years)"'))       
skills_junior_no_exp_df['percent_of_vacancies'] = skills_junior_no_exp_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and experience=="junior (no experince)"'))
skills_senior_df['percent_of_vacancies'] = skills_senior_df['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно" and experience=="senior (6+ years)"'))
skills['percent_of_vacancies'] = skills['vacancies'] / len(df.query('key_skills_from_key_skills_field != "неизвестно"'))


lists_for_plots = {
    'junior+ (1-3 years)':skills_junior_1_3_y_df,
    'middle (3-6 years)':skills_middle_3_6_y_df, 
    'junior (no experince)':skills_junior_no_exp_df,
    'senior (6+ years)':skills_senior_df
}
for keyword in lists_for_plots:
    fig = px.bar(lists_for_plots[keyword].head(15), x='skill', y='vacancies', width = 950, height = 450, color='skill', text='vacancies')
    fig.update_xaxes(tickangle=30)
    fig.update_layout(
        title='ТОП-15 ключевых навыков в профессии бизнес аналитик у {} '.format(keyword),
        xaxis_title="Ключевые навыки",
        yaxis_title="Количество вакансий с данным навыком")
    fig.update_traces(textposition='outside')
    for trace, percent in zip(fig.data, lists_for_plots[keyword].head(15)['percent_of_vacancies']):
        trace.name = trace.name.split('=')[0] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'


    fig.show()

## ТОП 15 работодателей 

In [38]:
df['employer_list'] = df['employer'].apply(lambda x: x.split(', '))
df

employer_list = []
skills_data_analyst = []
skills_business_analyst = []

lists = {
    'аналитик данных':skills_data_analyst,
    'бизнес аналитик':skills_business_analyst
}


for keyword in lists:
    for skills in df.query('category==@keyword')['employer_list']:
        for skill in skills:
            if skill != 'неизвестно':
                lists[keyword].append(skill)

employer_list = skills_data_analyst + skills_business_analyst
  

skills_data_analyst_df = pd.DataFrame({'skill':skills_data_analyst})
skills_business_analyst_df = pd.DataFrame({'skill':skills_business_analyst})
skills = pd.DataFrame({'skill':employer_list})

skills_data_analyst_df = pd.DataFrame(skills_data_analyst_df['skill'].value_counts()).reset_index()
skills_business_analyst_df = pd.DataFrame(skills_business_analyst_df['skill'].value_counts()).reset_index()
skills = pd.DataFrame(skills['skill'].value_counts()).reset_index()

skills_data_analyst_df.columns = ['skill', 'vacancies']
skills_business_analyst_df.columns = ['skill', 'vacancies']
skills.columns = ['skill', 'vacancies']

skills_data_analyst_df['percent_of_vacancies'] = skills_data_analyst_df['vacancies'] / len(df.query('employer != "неизвестно" and category=="аналитик данных"'))
skills_business_analyst_df['percent_of_vacancies'] = skills_business_analyst_df['vacancies'] / len(df.query('employer != "неизвестно" and category=="бизнес аналитик"'))
skills['percent_of_vacancies'] = skills['vacancies'] / len(df.query('employer != "неизвестно"'))

lists_for_plots = {
    'аналитик данных':skills_data_analyst_df,
    'бизнес аналитик':skills_business_analyst_df, 
    'аналитик данных / бизнес аналитик':skills
}

for keyword in lists_for_plots:
    fig = px.bar(lists_for_plots[keyword].head(15), x='skill', y='vacancies', width = 950, height = 450, color='skill', text='vacancies')
    fig.update_xaxes(tickangle=30)
    fig.update_layout(
        title='ТОП-15 работодателей в профессии {} '.format(keyword),
        xaxis_title="Работодатели",
        yaxis_title="количество вакансий")
    fig.update_traces(textposition='outside')
    for trace, percent in zip(fig.data, lists_for_plots[keyword].head(15)['percent_of_vacancies']):
        trace.name = trace.name.split('=')[0] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'


    fig.show()

## ТОП 15 городов России по предлагаемым вакансиям 

In [39]:
df['area_list'] = df['area'].apply(lambda x: x.split(', '))
df

area_list = []
skills_data_analyst = []
skills_business_analyst = []

lists = {
    'аналитик данных':skills_data_analyst,
    'бизнес аналитик':skills_business_analyst
}


for keyword in lists:
    for skills in df.query('category==@keyword')['area_list']:
        for skill in skills:
            if skill!='неизвестно':
                lists[keyword].append(skill)

skills_list = skills_data_analyst + skills_business_analyst
  

skills_data_analyst_df = pd.DataFrame({'skill':skills_data_analyst})
skills_business_analyst_df = pd.DataFrame({'skill':skills_business_analyst})
skills = pd.DataFrame({'skill':skills_list})

skills_data_analyst_df = pd.DataFrame(skills_data_analyst_df['skill'].value_counts()).reset_index()
skills_business_analyst_df = pd.DataFrame(skills_business_analyst_df['skill'].value_counts()).reset_index()
skills = pd.DataFrame(skills['skill'].value_counts()).reset_index()

skills_data_analyst_df.columns = ['skill', 'vacancies']
skills_business_analyst_df.columns = ['skill', 'vacancies']
skills.columns = ['skill', 'vacancies']

skills_data_analyst_df['percent_of_vacancies'] = skills_data_analyst_df['vacancies'] / len(df.query('area != "неизвестно" and category=="аналитик данных"'))
skills_business_analyst_df['percent_of_vacancies'] = skills_business_analyst_df['vacancies'] / len(df.query('area != "неизвестно" and category=="бизнес аналитик"'))
skills['percent_of_vacancies'] = skills['vacancies'] / len(df.query('area != "неизвестно"'))

lists_for_plots = {
    'аналитик данных':skills_data_analyst_df,
    'бизнес аналитик':skills_business_analyst_df, 
    'аналитик данных / бизнес аналитик':skills
}

for keyword in lists_for_plots:
    fig = px.bar(lists_for_plots[keyword].head(15), x='skill', y='vacancies', width = 950, height = 450, color='skill', text='vacancies')
    fig.update_xaxes(tickangle=30)
    fig.update_layout(
        title='ТОП-15 городов по профессии {} '.format(keyword),
        xaxis_title="название города",
        yaxis_title="количество вакансий")
    fig.update_traces(textposition='outside')
    for trace, percent in zip(fig.data, lists_for_plots[keyword].head(15)['percent_of_vacancies']):
        trace.name = trace.name.split('=')[0] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'


    fig.show()
  


   


In [40]:
df.columns

Index(['id', 'name', 'published_at', 'alternate_url', 'type', 'employer',
       'department', 'area', 'experience', 'key_skills', 'schedule',
       'employment', 'description', 'description_lemmatized', 'salary_from',
       'salary_to', 'salary_bin', 'key_skills_from_key_skills_field',
       'hard_skills_from_description', 'soft_skills_from_description',
       'category', 'skills_list', 'employer_list', 'area_list'],
      dtype='object')

## Типы предлагаемой занятости по вакансиям 

In [41]:
df['schedule_list'] = df['schedule'].apply(lambda x: x.split(', '))
df

schedule_list = []
skills_data_analyst = []
skills_business_analyst = []

lists = {
    'аналитик данных':skills_data_analyst,
    'бизнес аналитик':skills_business_analyst
}


for keyword in lists:
    for skills in df.query('category==@keyword')['schedule_list']:
        for skill in skills:
            if skill!='неизвестно':
                lists[keyword].append(skill)

schedule_list = skills_data_analyst + skills_business_analyst
  

skills_data_analyst_df = pd.DataFrame({'skill':skills_data_analyst})
skills_business_analyst_df = pd.DataFrame({'skill':skills_business_analyst})
skills = pd.DataFrame({'skill':schedule_list})

skills_data_analyst_df = pd.DataFrame(skills_data_analyst_df['skill'].value_counts()).reset_index()
skills_business_analyst_df = pd.DataFrame(skills_business_analyst_df['skill'].value_counts()).reset_index()
skills = pd.DataFrame(skills['skill'].value_counts()).reset_index()

skills_data_analyst_df.columns = ['skill', 'vacancies']
skills_business_analyst_df.columns = ['skill', 'vacancies']
skills.columns = ['skill', 'vacancies']

skills_data_analyst_df['percent_of_vacancies'] = skills_data_analyst_df['vacancies'] / len(df.query('schedule != "неизвестно" and category=="аналитик данных"'))
skills_business_analyst_df['percent_of_vacancies'] = skills_business_analyst_df['vacancies'] / len(df.query('schedule != "неизвестно" and category=="бизнес аналитик"'))
skills['percent_of_vacancies'] = skills['vacancies'] / len(df.query('schedule != "неизвестно"'))

lists_for_plots = {
    'аналитик данных':skills_data_analyst_df,
    'бизнес аналитик':skills_business_analyst_df, 
    'аналитик данных / бизнес аналитик':skills
}

for keyword in lists_for_plots:
    fig = px.bar(lists_for_plots[keyword].head(15), x='skill', y='vacancies', width = 950, height = 450, color='skill', text='vacancies')
    fig.update_xaxes(tickangle=30)
    fig.update_layout(
        title='Занятость по вакансии {} '.format(keyword),
        xaxis_title="название типа занятости",
        yaxis_title="количество вакансий")
    fig.update_traces(textposition='outside')
    for trace, percent in zip(fig.data, lists_for_plots[keyword].head(15)['percent_of_vacancies']):
        trace.name = trace.name.split('=')[0] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'


    fig.show()

**Выводы по исследовательскому анализу**

1. В ТОПе 15 ключевых навыков требуемых в профессии аналитик данных лидируют: sql, python, анализ данных, аналитическое мышление и power bi;
2. В ТОПе 15 ключевых навыков для бизнес аналитика лидируют: бизнес анализ, bpmn, аналитическое мышление, организация и моделирование бизнес процессов;
3. В зависимости от опыта в профессии аналитик данных требуются следующие навыки, перечислим по 5 наиболее востребованных:
 - junior+ (1-3 years): sql, python, анализ данных, аналитическое мышление и ms excel;
 - middle (3-6 years): sql, python, анализ данных, power bi и аналитическое мышление;
 - junior (no experience): sql, ms excel, python, ms powerpoint и аналитическое мышление;
 - senior : данные отсутствуют.
4. В зависимости от опыта в профессии бизнес аналитик требуются следующие навыки, перечислим по 5 наиболее востребованных:
 - junior+ (1-3 years): бизнес анализ, bpmn, организация бизнес процессов, аналитическое мышление и sql;
 - middle (3-6 years): бизнес анализ, bpmn, моделирование и оптимизация бизнес процессов и аналитическое мышление;
 - junior (no experience): бизнес анализ, моделирование бизнес процессов, sql, аналитический склад ума и bpmn;
 - senior: бизнес анализ, bpmn, sql, ms excel и start up project.
 5. По работодателям в разрезе профессии лидируют следующие компании:
 - аналитик данных: сбер, wildberries, t1, ozon и мтс;
 - бизнес аналитик: сбер, розничная сеть, магнит, банк ВТБ и т1.
 6. Среди городов лидируют:
 - аналитик данных: Москва, Санкт-Петербург, Казань, Екатеринбург и Нижний Новгород;
 - бизнес аналитик: Москва, Санкт-Петербург, Екатеринбург, Нижний Новгород и Казань.
 7. Наиболее восребованы следующие виды занятости:
 - аналитик данных: полный день(84%), удаленная работа(13,69%), гибкий график(1,68%) и сменный график(0,56%); 
 - бизнес аналитик: полный день(80,35%), удаленная работа(16,74%), гибкий график(2,81%) и сменный график(0,11%). 

 
 
 

# Выводы по проекту

На момент проведения анализа рынка всего в России по вакансиям аналитик данных и бизнес аналитик было 1284 открытых релевантных позиций в 640 компаниях. Из 1284 вакансий величина ЗП была указана в 142 случаях (11.06%)

Так как данные были представлены за один месяц исследование динамики не проводилось. Также в связи с наличием 89% пропусков в заработной плате, анализ заработной платы не проводился.

В зависимости от типа вакансии приоритеты ключевых навыков меняются, перечислим наиболее важные:
- аналитик данных: sql, python, анализ данных, аналитическое мышление и power bi;
- бизнес аналитик: бизнес анализ, bpmn, аналитическое мышление, организация и моделирование бизнес процессов.

В зависимости от опыта в профессии аналитик данных требуются следующие навыки:
- junior+ (1-3 years): sql, python, анализ данных, аналитическое мышление и ms excel;
- middle (3-6 years): sql, python, анализ данных, power bi и аналитическое мышление;
- junior (no experience): sql, ms excel, python, ms powerpoint и аналитическое мышление.
Можно сказать что для junior+ и middle востребованы примерно одинаковые навыки, а для junior (без опыта) желательно понимать еще ms excel и ms powerpoint.

В профессии бизнес аналитик в разрезе опыта востребованы:
- junior+ (1-3 years): бизнес анализ, bpmn, организация бизнес процессов, аналитическое мышление и sql;
- middle (3-6 years): бизнес анализ, bpmn, моделирование и оптимизация бизнес процессов и аналитическое мышление;
- junior (no experience): бизнес анализ, моделирование бизнес процессов, sql, аналитический склад ума и bpmn;
- senior: бизнес анализ, bpmn, sql, ms excel и start up project.

По работодателям в разрезе профессии лидируют следующие компании:
- аналитик данных: сбер, wildberries, t1, ozon и мтс;
- бизнес аналитик: сбер, розничная сеть, магнит, банк ВТБ и т1.

При исследовании вакансий в разрезе городов можно сказать, что лидируют одни и те же города:
- аналитик данных: Москва, Санкт-Петербург, Казань, Екатеринбург и Нижний Новгород;
- бизнес аналитик: Москва, Санкт-Петербург, Екатеринбург, Нижний Новгород и Казань.

При анализе требуемой занятости, мы видим, что лидирует с большим отрывом именно полная занятость, удаленный же тип занятости востребован относительно мало:
- аналитик данных: полный день(84%), удаленная работа(13,69%), гибкий график(1,68%) и сменный график(0,56%);
- бизнес аналитик: полный день(80,35%), удаленная работа(16,74%), гибкий график(2,81%) и сменный график(0,11%).

Далее рассмотрим подтвердились ли наши гипотезы.

1. Большую долю вакансий занимают вакансии на позицию Junior с опытом от года. 
Эта гипотеза подтвердилась полностью. 
- junior+ (1-3 years)      675
- middle (3-6 years)       505
- junior (no experince)     92
- senior (6+ years)         12

2. Москва лидирует по количеству вакансий.
Данная гипотеза 100% подтвердилась, это видно при исследовании требуемых вакансий в разрезе городов.

3. Предложений на удаленную работу больше чем на работу в офисе.
Гипотеза не подтвердилась. Большую долю занимают вакансии на полный день.
- полный день         1045
- удаленная работа     204
- гибкий график         32
- сменный график         3

Сформулируем рекомендации по навыкам для соискателя без опыта(junior no experince).

- Для поиска работы на должность аналитик данных необходимо знать: sql, python, ms excel, powerpoint и уметь мыслить аналитически. 

- На должность бизнес аналитик необходимо иметь следующие навыки: бизнес анализ, моделирование бизнес процессов, sql, аналитический склад ума и bpmn.


 

